# 2. 데이터 정제

## 데이터 불러오기

In [11]:
import pandas as pd
df = pd.read_csv("1_id_text.csv", encoding="utf-8")
df.head()

,search_keyword,date_created,time_created,writer,is_reply,text,id
0,JYP,2023-06-13,12:50:42,ㅇㅇ,0,jyp가반년만에2배가올랏네 근디보니는인적자원위주로하는사업은먼가ㅂㄹ다걍제조업이조음..,10100746844
1,JYP,2023-05-16,17:52:40,ㅇㅇ(118.34),0,야 개보지년들아 JYP 주가 봤냐.. 미쳣다 지금... 개잡주 오르는건 그러려니 하...,10100730087
2,JYP,2023-05-16,17:52:48,ㅇㅇ(118.34),1,ㅠㅠ,10100730087
3,JYP,2023-05-16,18:04:04,ㅇㅇ(203.227),1,ㅠㅠ,10100730087
4,JYP,2023-04-17,22:00:45,설묘.,0,니들 jyp 어케 생각해? ㄹㅇ 개미 너무 많이 붙는거 같은데,10100714904


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1153474 entries, 0 to 1153473
Data columns (total 7 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   search_keyword  1153474 non-null  object
 1   date_created    1153474 non-null  object
 2   time_created    1153474 non-null  object
 3   writer          1153474 non-null  object
 4   is_reply        1153474 non-null  int64 
 5   text            1153474 non-null  object
 6   id              1153474 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 61.6+ MB


## a. 무의미한 특수문자 제거

In [15]:
# 무의미한 특수문자 제거
import re
def remove_special_characters(input_text):
    # 정규 표현식을 사용하여 지정한 특수문자를 제외한 모든 특수문자를 찾아 제거
    pattern = r'[^\w.,!:()\[\]{}\-_@#$%^&\s]'  # \w는 문자와 숫자를 의미, \s는 공백을 의미
    cleaned_text = re.sub(pattern, '', input_text)
    return cleaned_text

In [20]:
# 테스트
temp_text = "ab■c!@#$^◆셀1트"
new_text = remove_special_characters(temp_text)
print(new_text)

abc!@#$^셀1트


In [21]:
!pip install soynlp

     ---------------------------------------- 0.0/416.8 kB ? eta -:--:--
      --------------------------------------- 10.2/416.8 kB ? eta -:--:--
     ------- ------------------------------- 81.9/416.8 kB 1.2 MB/s eta 0:00:01
     -------------------------------------- 416.8/416.8 kB 4.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.2 MB ? eta -:--:--
      --------------------------------------- 0.6/42.2 MB 12.6 MB/s eta 0:00:04
     - -------------------------------------- 1.1/42.2 MB 12.0 MB/s eta 0:00:04
     - -------------------------------------- 1.6/42.2 MB 11.6 MB/s eta 0:00:04
     -- ------------------------------------- 2.2/42.2 MB 11.6 MB/s eta 0:00:04
     -- ------------------------------------- 2.7/42.2 MB 11.5 MB/s eta 0:00:04
     --- ------------------------------------ 3.3/42.2 MB 11.5 MB/s eta 0:00:04
     --- ------------------------------------ 3.8/42.2 MB 12.0 MB/s eta 0:00:04
     ---- ----------------------------------- 4.5/42.2 MB 12.

## b. 반복되는 문자 최대 3개로 줄임

In [29]:
# 반복되는 문자를 최대 3개만 남깁니다
from soynlp.normalizer import *
temp_text = "zzzz ㅋ ㅠㅠㅠㅠ ㅋㅋ 1234 ㅋㅋㅋ 하하하하 ㅋㅋㅋㅋㅋ "
new_text = emoticon_normalize(temp_text, num_repeats=3)
print(new_text)

zzz ㅋ ㅠㅠㅠ ㅋㅋ 1234 ㅋㅋㅋ 하하하 ㅋㅋㅋ


## text 정제 함수 : a + b

In [32]:
# text 정제 함수
def cleanse_text(input_text):
    cleansed_text = remove_special_characters(input_text)
    cleansed_text = emoticon_normalize(cleansed_text, num_repeats=3)
    return cleansed_text

In [33]:
# 테스트
temp_text = "b■c!@#$^◆ㅋㅋㅋㅋㅋㅋㅋㅋ"
new_text = cleanse_text(temp_text)
print(new_text)

bc!@#$^ㅋㅋㅋ


## df에 적용하기

In [34]:
# 모든 text를 정제한다.
df['cleansed_text'] = df['text'].apply(cleanse_text)
df.tail(3)

,search_keyword,date_created,time_created,writer,is_reply,text,id,cleansed_text
1153471,휴젤,2020-04-29,11:55:10,ㅇㅇ(123.215),0,미국주식보다 한국꺼보면 ㄹㅇ 암걸릴 것 같음 특히 휴젤 저거 아는분이 임원진인데 회...,241400125127,미국주식보다 한국꺼보면 ㄹㅇ 암걸릴 것 같음 특히 휴젤 저거 아는분이 임원진인데 회...
1153472,휴젤,2020-04-29,11:56:16,ㅇㄷ(175.223),1,한국 상장회사들중에 ㅂㅅ같은곳이 넘 많음...,241400125127,한국 상장회사들중에 ㅂㅅ은곳이 넘 많음...
1153473,휴젤,2020-04-29,11:56:38,ㅇㅇ(223.33),1,신흥국 회사투자 망설여지는 게 이거 때문임. 주식시장에 대한 개념이 안 잡혀있음.,241400125127,신흥국 회사투자 망설여지는 게 이거 때문임. 주식시장에 대한 개념이 안 잡혀있음.


In [5]:
# 결과 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1153474 entries, 0 to 1153473
Data columns (total 8 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   search_keyword  1153474 non-null  object
 1   date_created    1153474 non-null  object
 2   time_created    1153474 non-null  object
 3   writer          1153474 non-null  object
 4   is_reply        1153474 non-null  int64 
 5   text            1153474 non-null  object
 6   id              1153474 non-null  int64 
 7   cleansed_text   1152312 non-null  object
dtypes: int64(2), object(6)
memory usage: 70.4+ MB


## 결측치 제거

In [7]:
# 결측치 제거
df_dropna = df.dropna().copy()
df_dropna.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1152312 entries, 0 to 1153473
Data columns (total 8 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   search_keyword  1152312 non-null  object
 1   date_created    1152312 non-null  object
 2   time_created    1152312 non-null  object
 3   writer          1152312 non-null  object
 4   is_reply        1152312 non-null  int64 
 5   text            1152312 non-null  object
 6   id              1152312 non-null  int64 
 7   cleansed_text   1152312 non-null  object
dtypes: int64(2), object(6)
memory usage: 79.1+ MB


## 파일로 저장

In [8]:
# .csv 파일로 저장한다
df_dropna.to_csv("2_cleansed_text.csv", index=False, encoding="utf-8")

In [9]:
# 말뭉치를 .txt 파일로 저장한다
texts = df_dropna['cleansed_text'].tolist()

with open('2_cleansed_corpus.txt', 'w', encoding='utf-8') as file:
    for text in texts:
        file.write(text + '\n')